<a href="https://colab.research.google.com/github/Y00nnye/Teamproject_23-2/blob/douen/_2_bert%2Bvoting%2Bevaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1.bert사용
#bert_keyword초기화
bert_keyword=[]

# X_kiwi_combined의 길이만큼 반복: 모든 단어의 중요도 출력
for i in range(len(X_kiwi_combined)):
    # 키워드 추출
    keywords = kw_model.extract_keywords(X_kiwi_combined[i], keyphrase_ngram_range=(1, 1), stop_words=None, top_n=1)

    # bert_keyword 에 키워드 추가
    bert_keyword.append(keywords)

for i, value in enumerate(bert_keyword):
    print(f"value at index {i}: {value}")

#위의 bert_keyword df화

import pandas as pd

# 주어진 데이터:bert_keyword

# DataFrame으로 변환
df = pd.DataFrame([(item[0], item[1]) for sublist in bert_keyword for item in sublist], columns=['단어', '중요도'])

# 결과 출력
display(df)

def pad_sequence(sequence, max_length, padding_value=0):
    padded_sequence = sequence[:max_length] + [padding_value] * max(0, max_length - len(sequence))
    return padded_sequence

# 최대 길이
max_length = 20

# 패딩된 결과
padded_X_kiwi = [pad_sequence(row, max_length) for row in X_kiwi]

# 결과 출력
for row in padded_X_kiwi:
    print(row)

df_X = pd.DataFrame(padded_X_kiwi)

# 결과 출력
print(df_X)

df_keyword = pd.DataFrame(bert_keyword)

# 튜플 분해 및 새로운 열 생성
df_keyword[['단어', '중요도']] = pd.DataFrame(df_keyword[0].tolist(), index=df_keyword.index)

# 기존 열 삭제
df_keyword = df_keyword.drop(columns=[0])

# 결과 출력
print(df_keyword)

# df와 bert_keyword를 수평으로 연결
result_df = pd.concat([df_X,df_keyword], axis=1)

# 결과 출력
display(result_df)

In [ ]:
#2.hardvoting(임의의 thresholds설정 후 직접 구현)

#hardvoting직접구현(중요도 값=> 0.6 0.7 0.8처럼 임의로 기준치 설정후=>hardvoting직접구현)

from sklearn.dummy import DummyClassifier

# 중요도 기준 설정
thresholds = [0.5, 0.6, 0.7]

# 투표 수집
votes = []

for threshold in thresholds:
    # 중요도 기준에 따라 투표
    votes.append([1 if importance >= threshold else 0 for importance in df_keyword['중요도']])

# vote를 label로 변환
labels = [1 if sum(vote) > len(vote) / 2 else 0 for vote in zip(*votes)]

# 중간 결과 출력
print("중간 투표 결과:")
for threshold, vote in zip(thresholds, votes):
    print(f"중요도 기준 {threshold}에 대한 투표 결과: {vote}")

print("labels")
print(labels)

# 최종 키워드 여부 결정
df_keyword['키워드 여부'] = labels

# 최종 결과 DataFrame 생성
result_df = df_keyword[['단어', '중요도', '키워드 여부']]

# 최종 결과 출력
print("\n최종 결과:")
display(result_df)

In [ ]:
#3.hard,soft voting(3different classifier)

#classifier:logistic, knn, decision tree







#(1)로지스틱 회귀 계산
#soft Voting Classifier(로지스틱 회귀)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(df_keyword['중요도'].values.reshape(-1, 1),
                                                    labels, test_size=0.2, random_state=42)

# Logistic Regression 모델 학습
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Logistic Regression 모델의 테스트 데이터에 대한 예측 확률
logistic_predictions_proba = logistic_model.predict_proba(X_test)

# 결과 출력
print("Logistic Regression 모델의 예측 확률:")
print(logistic_predictions_proba)
print(logistic_predictions_proba.shape)









#(2)softvotingfrom sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split

# 데이터 분리
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(
    df_keyword['중요도'].values.reshape(-1, 1),
    labels,
    df_keyword.index,  # 인덱스를 따로 저장
    test_size=0.2,
    random_state=42
)

# Voting Classifier에 사용할 분류기들 정의
classifier1 = KNeighborsClassifier(n_neighbors=3)
classifier2 = DecisionTreeClassifier(random_state=42)
classifier3 = LogisticRegression()

# Voting Classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('knn', classifier1),       # KNeighborsClassifier
        ('tree', classifier2),      # DecisionTreeClassifier
        ('logistic', classifier3)   # LogisticRegression
    ],
    voting='soft',  # Soft Voting 설정
    weights=[1, 1, 1]  # 각 분류기에 가중치 부여
)

# Voting Classifier 모델 학습
voting_classifier.fit(X_train, y_train)

# KNeighborsClassifier 모델 학습
classifier1.fit(X_train, y_train)

# 각 분류기의 예측 확률을 가져옵니다.
knn_predictions_proba = classifier1.predict_proba(X_test)

# Voting Classifier 모델 학습
voting_classifier.fit(X_train, y_train)


# 각 분류기에 대해 fit 메소드 호출
classifier1.fit(X_train, y_train)
classifier2.fit(X_train, y_train)
classifier3.fit(X_train, y_train)

# 각 분류기의 예측 확률을 가져옵니다.
knn_predictions_proba = classifier1.predict_proba(X_test)
tree_predictions_proba = classifier2.predict_proba(X_test)
logistic_predictions_proba = classifier3.predict_proba(X_test)

# 각 분류기의 예측 확률을 가져옵니다.
knn_predictions_proba = classifier1.predict_proba(X_test)
tree_predictions_proba = classifier2.predict_proba(X_test)
logistic_predictions_proba = classifier3.predict_proba(X_test)

# Soft Voting의 결과를 가져옵니다.
soft_voting_predictions_proba = voting_classifier.predict_proba(X_test)

import pandas as pd

# 각 분류기의 예측 확률을 데이터프레임으로 변환
knn_proba_df = pd.DataFrame(knn_predictions_proba, columns=['KNeighborsClassifier_0', 'KNeighborsClassifier_1'])
tree_proba_df = pd.DataFrame(tree_predictions_proba, columns=['DecisionTreeClassifier_0', 'DecisionTreeClassifier_1'])
logistic_proba_df = pd.DataFrame(logistic_predictions_proba, columns=['LogisticRegression_0', 'LogisticRegression_1'])
voting_proba_df = pd.DataFrame(voting_predictions_proba, columns=['SoftVotingClassifier_0', 'SoftVotingClassifier_1'])

# 예측 확률 데이터프레임을 수평으로 결합
combined_proba_df = pd.concat([knn_proba_df, tree_proba_df, logistic_proba_df, voting_proba_df], axis=1)

# 결과 출력
display(combined_proba_df)

#anntation(plotly)

import plotly.graph_objects as go

def plot_probabilities_plotly(probabilities, classifier_name):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=list(range(len(probabilities))),
                             y=probabilities[:, 1],
                             mode='lines+markers',
                             name=f'{classifier_name} - Class 1 Probability'))

    fig.update_layout(title=f'{classifier_name} Predicted Probabilities',
                      xaxis_title='test Index',
                      yaxis_title='Probability')

    fig.show()

# 각 분류기의 예측 확률을 plotly로 시각화
plot_probabilities_plotly(knn_predictions_proba, 'KNeighborsClassifier')
plot_probabilities_plotly(tree_predictions_proba, 'DecisionTreeClassifier')
plot_probabilities_plotly(logistic_predictions_proba, 'LogisticRegression')
plot_probabilities_plotly(voting_predictions_proba, 'SoftVotingClassifier')





#(3)hard voting(classifier:logistic+knn+desicion tree)

# Voting Classifier에 사용할 분류기들 정의
classifier1 = KNeighborsClassifier(n_neighbors=3)
classifier2 = DecisionTreeClassifier(random_state=42)
classifier3 = LogisticRegression()

# Hard Voting Classifier
hard_voting_classifier = VotingClassifier(
    estimators=[
        ('knn', classifier1),       # KNeighborsClassifier
        ('tree', classifier2),      # DecisionTreeClassifier
        ('logistic', classifier3)   # LogisticRegression
    ],
    voting='hard'  # Hard Voting 설정
)

# Voting Classifier 모델 학습
hard_voting_classifier.fit(X_train, y_train)

# 각 분류기에 대해 fit 메소드 호출
classifier1.fit(X_train, y_train)
classifier2.fit(X_train, y_train)
classifier3.fit(X_train, y_train)

# 각 분류기의 예측을 가져옵니다.
knn_predictions = classifier1.predict(X_test)
tree_predictions = classifier2.predict(X_test)
logistic_predictions = classifier3.predict(X_test)

# Hard Voting의 결과를 가져옵니다.
hard_voting_predictions = hard_voting_classifier.predict(X_test)

# 결과 출력
result_df_hard_voting = pd.DataFrame({
    '중요도': df_keyword.loc[indices_test, '중요도'],
    'KNeighborsClassifier': knn_predictions,
    'DecisionTreeClassifier': tree_predictions,
    'LogisticRegression': logistic_predictions,
    'HardVoting': hard_voting_predictions
})

display(result_df_hard_voting)

#annotation(plotpy)

import plotly.graph_objects as go

def plot_predictions_plotly(predictions, classifier_name):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=list(range(len(predictions))),
                             y=predictions,
                             mode='lines+markers',
                             name=f'{classifier_name} Predictions'))

    fig.update_layout(title=f'{classifier_name} Predictions',
                      xaxis_title='test Index',
                      yaxis_title='Predictions')

    fig.show()

# 각 분류기의 예측을 plotly로 시각화
plot_predictions_plotly(knn_predictions, 'KNeighborsClassifier')
plot_predictions_plotly(tree_predictions, 'DecisionTreeClassifier')
plot_predictions_plotly(logistic_predictions, 'LogisticRegression')
plot_predictions_plotly(hard_voting_predictions, 'HardVotingClassifier')



In [ ]:
#4.성능 비교
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Soft Voting의 성능 평가
soft_voting_accuracy = accuracy_score(y_test, soft_voting_predictions)
soft_voting_precision = precision_score(y_test, soft_voting_predictions)
soft_voting_recall = recall_score(y_test, soft_voting_predictions)
soft_voting_f1 = f1_score(y_test, soft_voting_predictions)

# Hard Voting의 성능 평가
hard_voting_accuracy = accuracy_score(y_test, hard_voting_predictions)
hard_voting_precision = precision_score(y_test, hard_voting_predictions)
hard_voting_recall = recall_score(y_test, hard_voting_predictions)
hard_voting_f1 = f1_score(y_test, hard_voting_predictions)

# 결과 출력
print("Soft Voting 성능:")
print(f"정확도: {soft_voting_accuracy:.4f}")
print(f"정밀도: {soft_voting_precision:.4f}")
print(f"재현율: {soft_voting_recall:.4f}")
print(f"F1 점수: {soft_voting_f1:.4f}")

print("\nHard Voting 성능:")
print(f"정확도: {hard_voting_accuracy:.4f}")
print(f"정밀도: {hard_voting_precision:.4f}")
print(f"재현율: {hard_voting_recall:.4f}")
print(f"F1 점수: {hard_voting_f1:.4f}")
